In [1]:
from tool.classes import *
from tool.general import *
from tool.download import *
#from tool.preprocess import *
#from tool.build import *
#from tool.search import *

In [2]:
download_all_bidixes()

2018-06-10 22:36:52,896 | INFO : Start
2018-06-10 22:47:16,453 | INFO : Finish


For now I delete ces-ces, ita-ita, eus-eu_bis, hbs_HR-hbs_SR, zh_CN-zh_TW

In [1]:
from tool.classes import *
from tool.general import *
from tool.preprocess import *

In [3]:
os

<module 'os' from 'C:\\Users\\Glaz\\Anaconda3\\lib\\os.py'>

In [2]:
%time monodix()

2018-06-10 23:08:09,965 | INFO : started
2018-06-10 23:11:22,982 | INFO : finished
Wall time: 3min 13s


In [3]:
%time get_relevant_languages('spa', 'eng')

Wall time: 40 ms


In [2]:
load_file('spa', 'eng')

2018-06-10 23:15:20,771 | INFO : cat-ita started


NameError: name 'DiGetItem' is not defined